In [1]:
import keras,os
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

imageShape=(224,224,3)
imageSize=(224,224)

2023-05-03 00:53:03.758699: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(directory='/Users/krishna/Documents/Projects/Smart_Mirror/clothes',target_size=imageSize)
tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(directory='/Users/krishna/Documents/Projects/Smart_Mirror/test', target_size=imageSize)

Found 4862 images belonging to 20 classes.
Found 541 images belonging to 20 classes.


In [3]:

from keras.applications.vgg16 import VGG16
#model = VGG16(include_top=False,input_shape=(32,32,3),weights='imagenet',classifier_activation='softmax',classes=20)

vgg = VGG16(input_shape=imageShape, weights='imagenet', include_top=False, pooling='avg')
vgg.trainable = False
model = Sequential()
model.add(vgg)

model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=20, activation="sigmoid"))



2023-05-03 00:53:08.700848: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [4]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 512)               14714688  
                                                                 
 flatten (Flatten)           (None, 512)               0         
                                                                 
 dense (Dense)               (None, 4096)              2101248   
                                                                 
 dense_1 (Dense)             (None, 4096)              16781312  
                                                                 
 dense_2 (Dense)             (None, 20)                81940     
                                                                 
Total params: 33,679,188
Trainable params: 18,964,500
Non-trainable params: 14,714,688
_________________________________________________________________


In [5]:
from keras.optimizers import SGD
opt = SGD(learning_rate=0.001)
model.compile(optimizer=opt, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [6]:
#  hist = model.load_weights("/Users/krishna/Documents/Projects/Smart_Mirror/vgg16_1.h5")


In [7]:
from keras.callbacks import ModelCheckpoint, EarlyStopping
checkpoint = ModelCheckpoint("/Users/krishna/Documents/Projects/Smart_Mirror/vgg16_1.h5", monitor='val_accuracy', verbose=1, save_best_only=True, save_weights_only=False, mode='auto', save_freq='epoch')
early = EarlyStopping(monitor='val_accuracy', min_delta=0, patience=20, verbose=1, mode='auto')
hist = model.fit_generator(generator=traindata, validation_data= testdata, validation_steps=10,epochs=20,callbacks=[checkpoint,early])

/var/folders/08/_ldbqvz16j10d6w1jc1yxknr0000gn/T/ipykernel_83709/2336221160.py:4: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  hist = model.fit_generator(generator=traindata, validation_data= testdata, validation_steps=10,epochs=20,callbacks=[checkpoint,early])


Epoch 1/20
  1/152 [..............................] - ETA: 18:32 - loss: 4.6039 - accuracy: 0.1875

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
# plt.plot(hist.history["accuracy"])
# plt.plot(hist.history['val_accuracy'])
# plt.plot(hist.history['loss'])
# plt.plot(hist.history['val_loss'])
# plt.title("model accuracy")
# plt.ylabel("Accuracy")
# plt.xlabel("Epoch")
# plt.legend(["Accuracy","Validation Accuracy","loss","Validation Loss"])
# plt.show()

In [ ]:
Labels = [['Not sure',0],[
 'T-Shirt', 1],[
 'Shoes', 2],[
 'Shorts', 3],[
 'Shirt', 4],[
 'Pants', 5],[
 'Skirt', 6],[
 'Other', 7],[
 'Top', 8],[
 'Outwear', 9],[
 'Dress', 10],[
 'Body', 11],[
 'Longsleeve', 12],[
 'Undershirt', 13],[
 'Hat', 14],[
 'Polo', 15],[
 'Blouse', 16],[
 'Hoodie', 17],[
 'Skip', 18],[
 'Blazer', 19]
    
]

In [ ]:
from keras.utils import load_img
img = keras.utils.load_img("/Users/krishna/Documents/Projects/Smart_Mirror/archive/trial/trial9.jpg",target_size=(224,224))
img = np.asarray(img)
plt.imshow(img)
img = np.expand_dims(img, axis=0)
from keras.models import load_model
saved_model = load_model("vgg16_1.h5")
output = saved_model.predict(img)
answer = output.argmax()
print(answer)

for i in range(len(Labels)):
    if Labels[i][1] == answer:
        print(Labels[i][0])

